<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=04ad52ff6c90dab063a6e368c17f2313c800e39398032a47739590a7a360fb21
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 12:22:41
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 10.46 K (0.07%)
Current PnL: -17.68 L (-11.89%)
CY Booked + Current PnL: -6.65 L (-4.48%)
-------------------
Total profit:  2.05 L
Total loss:  -19.74 L
-------------------
Total Booked + Current PnL: 20.33 L (16.7%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.79 L (59.21%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.20,60.0,H-LC,11.19,173382.0,17205.0,8894.0,0.02,11.02,5.13,16.71,22.0,1.93,1.26,30.38,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,58.0,M-LC,4.26,217698.0,16218.0,10994.0,-0.26,8.05,5.05,13.51,60.0,1.48,1.59,21.66,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,103.42,57.0,M-SC,4.99,87639.0,-13138.0,13233.0,-0.62,-13.04,15.10,0.09,245.0,-0.99,0.64,15.14,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.24,114772.0,14268.0,16366.0,1.00,14.20,14.26,30.49,79.0,0.87,0.84,51.20,MH,ATH,METALS
41,ITC,452.00,-37.08,57.0,H-LC,2.97,203106.0,2968.0,17914.0,0.07,1.48,8.82,10.44,4.0,0.17,1.48,7.13,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.94,226185.0,1223.0,166427.0,-0.36,0.54,73.58,74.53,192.0,0.01,1.65,32.97,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.22,41.0,H-MC,2.08,135600.0,-188.0,42945.0,0.26,-0.14,31.67,31.48,107.0,-0.00,0.99,11.96,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,59.0,H-MC,5.59,179546.0,-262.0,112665.0,-1.15,-0.15,62.75,62.52,88.0,-0.00,1.31,33.89,XR,NTT,BANKS
51,PGHH,17973.08,-31.21,49.0,H-MC,4.54,199080.0,-1740.0,70514.0,-0.25,-0.87,35.42,34.25,80.0,-0.02,1.45,4.29,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299828.0,-16014.0,128686.0,-0.99,-5.07,42.92,35.67,5.0,-0.12,2.18,8.08,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.39,44.0,H-LC,13.52,267847.0,482.0,64980.0,0.24,0.18,24.26,24.48,48.0,0.01,1.95,55.25,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.37,45.0,H-LC,8.07,224288.0,-9856.0,58472.0,-1.14,-4.21,26.07,20.77,86.0,-0.17,1.63,2.42,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,230272.0,-11652.0,74931.0,-0.07,-4.82,32.54,26.15,8.0,-0.16,1.68,8.82,X40,ATH,IT
1,ABB,7934.00,-40.06,53.0,H-LC,11.96,250560.0,-11059.0,130266.0,0.04,-4.23,51.99,45.57,7.0,-0.08,1.82,6.62,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.47,59.0,H-LC,1.19,239085.0,-7066.0,86286.0,0.41,-2.87,36.09,32.18,16.0,-0.08,1.74,32.14,X200,ATH,IT
41,ITC,452.00,-37.08,57.0,H-LC,2.97,203106.0,2968.0,17914.0,0.07,1.48,8.82,10.44,4.0,0.17,1.48,7.13,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,3.55,284825.0,19399.0,37398.0,-0.29,7.31,13.13,21.40,11.0,0.52,2.07,15.17,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.09,56.0,H-LC,5.91,215679.0,11179.0,30238.0,0.86,5.47,14.02,20.25,10.0,0.37,1.57,14.68,X40N,BTT,PHARMA
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299828.0,-16014.0,128686.0,-0.99,-5.07,42.92,35.67,5.0,-0.12,2.18,8.08,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3981.08,67.0,M-SC,7.14,127204.0,-20164.0,71985.0,11.23,-13.68,56.59,35.16,236.0,-0.28,0.93,29.93,XY24,NTT,AUTO
14,BERGEPAINT,680.00,-16.95,55.0,H-MC,6.79,223297.0,-4042.0,52095.0,2.93,-1.78,23.33,21.14,106.0,-0.08,1.63,25.86,XY24,NTT,PAINTS
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.40,101370.0,9872.0,68881.0,1.62,10.79,67.95,86.07,223.0,0.14,0.74,47.21,XR,NTT,DURABLES
33,HONAUT,58357.33,-20.66,49.0,H-SC,2.66,111765.0,-16173.0,63304.0,1.53,-12.64,56.64,36.84,143.0,-0.26,0.81,14.42,X40N,ATH,ELECTRICAL
58,ROUTE,2236.84,-46.55,47.0,H-SC,18.05,87683.0,-45383.0,138241.0,1.44,-34.11,157.66,69.78,118.0,-0.33,0.64,2.38,SR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.0,242.38,70.0,M-SC,5.43,152744.0,-30334.0,109319.0,-2.84,-16.57,71.57,43.14,230.0,-0.28,1.11,43.68,XY24,NTT,MISC
53,RAJESHEXPO,518.0,1819.19,61.0,L-SC,5.57,52377.0,-84800.0,84893.0,-2.39,-61.82,162.08,0.07,267.0,-1.00,0.38,30.26,OX40N,NTT,JEWELLERY
69,SURYODAY,240.0,40.35,50.0,H-SC,3.27,129010.0,-50061.0,115786.0,-1.61,-27.96,89.75,36.71,135.0,-0.43,0.94,28.00,XR,NTT,BANKS
24,DABUR,735.0,-4.02,54.0,H-MC,6.14,208904.0,11876.0,77754.0,-1.55,6.03,37.22,45.49,102.0,0.15,1.52,19.85,XY24,BTT,FMCG
64,SHALBY,327.0,1057.89,67.0,M-SC,2.01,157484.0,-23659.0,69120.0,-1.45,-13.06,43.89,25.10,235.0,-0.34,1.15,26.21,XY24,NTT,HEALTHCARE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,62.0,H-SC,12.58,126927.0,-15552.0,35070.0,-0.49,-10.92,27.63,13.70,163.0,-0.44,0.92,49.81,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,103.42,57.0,M-SC,4.99,87639.0,-13138.0,13233.0,-0.62,-13.04,15.10,0.09,245.0,-0.99,0.64,15.14,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,61.0,H-SC,2.00,224280.0,-45387.0,81728.0,1.05,-16.83,36.44,13.47,138.0,-0.56,1.63,13.44,XY24,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.14,146016.0,-29887.0,71913.0,0.11,-16.99,49.25,23.89,149.0,-0.42,1.06,26.10,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.35,52.0,H-MC,7.98,107844.0,-23991.0,66216.0,-1.33,-18.20,61.40,32.03,98.0,-0.36,0.79,20.79,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.40,294361.0,-51595.0,175439.0,0.23,-14.91,59.60,35.80,1.0,-0.29,2.14,4.28,X40,BTT,IT
39,INFY,2275.00,-16.37,56.0,H-LC,9.45,325208.0,12320.0,159352.0,1.13,3.94,49.00,54.87,3.0,0.08,2.37,10.85,X40,BTT,IT
41,ITC,452.00,-37.08,57.0,H-LC,2.97,203106.0,2968.0,17914.0,0.07,1.48,8.82,10.44,4.0,0.17,1.48,7.13,X40,NTT,FMCG
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299828.0,-16014.0,128686.0,-0.99,-5.07,42.92,35.67,5.0,-0.12,2.18,8.08,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,11.96,250560.0,-11059.0,130266.0,0.04,-4.23,51.99,45.57,7.0,-0.08,1.82,6.62,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,63.0,M-MC,6.94,226185.0,1223.0,166427.0,-0.36,0.54,73.58,74.53,192.0,0.01,1.65,32.97,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.17,53.0,H-MC,3.96,188550.0,23880.0,44083.0,-0.07,14.50,23.38,41.28,89.0,0.54,1.37,23.64,X40N,ATH,AC
84,VOLTAS,1918.49,0.18,56.0,H-MC,2.97,210090.0,18348.0,77691.0,0.38,9.57,36.98,50.08,99.0,0.24,1.53,17.02,XY25,ATH,AC
24,DABUR,735.00,-4.02,54.0,H-MC,6.14,208904.0,11876.0,77754.0,-1.55,6.03,37.22,45.49,102.0,0.15,1.52,19.85,XY24,BTT,FMCG
18,CAMS,5211.76,-5.38,50.0,H-SC,4.75,107514.0,5510.0,38415.0,-0.28,5.40,35.73,43.06,122.0,0.14,0.78,23.44,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.24,114772.0,14268.0,16366.0,1.00,14.20,14.26,30.49,79.0,0.87,0.84,51.20,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,56.0,M-SC,1.40,101370.0,9872.0,68881.0,1.62,10.79,67.95,86.07,223.0,0.14,0.74,47.21,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.53,54.0,H-SC,12.26,138155.0,14819.0,118938.0,-0.22,12.02,86.09,108.45,119.0,0.12,1.01,37.05,AR,ATH,MISC
86,WIPRO,420.00,-11.07,53.0,M-LC,5.81,156085.0,5140.0,104312.0,-0.85,3.41,66.83,72.51,53.0,0.05,1.14,9.63,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,59.0,H-MC,5.59,179546.0,-262.0,112665.0,-1.15,-0.15,62.75,62.52,88.0,-0.00,1.31,33.89,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.53,86976.0,-26573.0,66624.0,-0.81,-23.40,76.60,35.27,268.0,-0.40,0.63,112.88,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.51,43.0,H-SC,14.00,87952.0,-13011.0,107539.0,-0.22,-12.89,122.27,93.63,148.0,-0.12,0.64,31.00,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.44,56.0,H-SC,13.13,94845.0,-3135.0,24736.0,0.08,-3.20,26.08,22.05,152.0,-0.13,0.69,37.86,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.24,114772.0,14268.0,16366.0,1.00,14.20,14.26,30.49,79.0,0.87,0.84,51.20,MH,ATH,METALS
82,VALIANTORG,838.00,-323.33,53.0,H-SC,7.78,124100.0,-39505.0,160821.0,0.00,-24.15,129.59,74.15,139.0,-0.25,0.90,59.08,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3981.08,67.0,M-SC,7.14,127204.0,-20164.0,71985.0,11.23,-13.68,56.59,35.16,236.0,-0.28,0.93,29.93,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.53,86976.0,-26573.0,66624.0,-0.81,-23.40,76.60,35.27,268.0,-0.40,0.63,112.88,XR,NTT,HOTELS
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.24,114772.0,14268.0,16366.0,1.00,14.20,14.26,30.49,79.0,0.87,0.84,51.20,MH,ATH,METALS
71,TANLA,1963.11,-19.92,73.0,H-SC,9.26,214965.0,-55796.0,385755.0,-0.23,-20.61,179.45,121.86,133.0,-0.14,1.57,68.16,AR,ATH,IT
55,RELAXO,1176.00,-32.23,57.0,H-SC,2.20,92317.0,-52843.0,128773.0,-0.64,-36.40,139.49,52.31,136.0,-0.41,0.67,23.57,X40N,NTT,FOOTWEAR


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,43.51
2,50,73.85


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.52
LC    34.49
MC    23.00
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.99
X40      15.30
X40N     12.94
XR       10.01
XY25      9.75
AR        8.64
OX40N     8.20
X200      1.74
MH        1.73
X5K       1.43
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.39
H-LC    26.28
H-MC    20.06
M-SC    13.58
M-LC     7.15
M-MC     2.63
L-SC     1.55
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.43,-14.09,78.95
FMCG,11.84,-4.05,40.46
FINANCE,9.61,-16.53,62.18
BANKS,7.78,-12.35,64.07
MISC,6.08,-13.77,78.71
PAINTS,5.93,-13.37,38.75
ELECTRICAL,5.58,-7.31,66.45
AC,3.61,2.28,30.81
INSURANCE,3.39,-8.13,45.22


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2865658.0,22
AR,1311034.0,9
XR,1238420.0,13
X40,866982.0,10
X40N,632225.0,10
OX40N,520699.0,10
XY25,411078.0,7
SR,245780.0,2
X5K,121836.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3353630.0,28
H-LC,1381603.0,15
M-SC,1381095.0,17
H-MC,1200723.0,15
M-LC,398315.0,5
M-MC,316441.0,2
L-SC,248052.0,3
L-MC,60198.0,1
L-LC,38714.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1041489.0      6
M-SC       XY24       794319.0      7
H-SC       AR         780234.0      5
           XR         682273.0      6
H-LC       X40        576795.0      5
H-MC       XY24       487256.0      4
H-LC       AR         368311.0      2
M-MC       XY24       316441.0      2
H-SC       X40N       296260.0      4
           SR         245780.0      2
           OX40N      245187.0      4
H-LC       X40N       226290.0      4
H-MC       X40        220072.0      4
           XY25       188473.0      2
L-SC       XR         163159.0      2
M-SC       AR         162489.0      2
M-LC       XY24       161173.0      2
M-SC       OX40N      124403.0      4
M-LC       X5K        121836.0      1
M-SC       XR         115813.0      2
           XY25       113956.0      1
H-MC       XR         112665.0      1
           X40N       109675.0      2
M-LC       XR         104312.0      1
H-LC       X200        86286.0      1
L-SC       OX40N       84893.0      1
M-SC       X40         70115.0      1
H-MC       OX40N       66216.0      1
H-LC       XY24        64980.0      1
H-SC       MH          62407.0      1
L-MC       XR          60198.0      1
H-LC       XY25        58941.0      2
L-LC       XY25        38714.0      1
H-MC       MH          16366.0      1
M-LC       XY25        10994.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
